<br>
<br>
<br>

আগের ভিডিও তে আমরা SimpleImputer(strategy="") ব্যবহার করে যত গুলো মিসিং ভ্যালু ছিল সব গুলোকেই fill করেছিলাম কিন্তু আমাদের যদি কিছু সংখ্যক মিসিং ভ্যালু তে অন্য strategy apply করতে চাই তাইলে কিভাবে করবো আমরা?
<br>
<br>
<br>
এর জন্য আমরা ৩টা  sklearn এর ক্লাস ব্যবহার করবোঃ 

-  SimpleImputer

-   Pipeline

-   ColumnTransformer


In [1]:
# import library and classes: 
import numpy as np 
import pandas as pd 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
# Load the dataset:
train = pd.read_csv("dataSet/Hourse_Prices/train.csv")
test = pd.read_csv("dataSet/Hourse_Prices/test.csv")

print(f"train dataset Shape: {train.shape}")
print(f"test dataset Shape: {test.shape}")

train dataset Shape: (1460, 81)
test dataset Shape: (1459, 80)


In [3]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [6]:
# convert the dataset into train dataset (see_lec12 for more details):

X_train = train.drop(columns="SalePrice")
y_train = train["SalePrice"]
X_test = test.copy()

print(f"X_train dataset Shape: {X_train.shape}")
print(f"y_train dataset Shape: {y_train.shape}")
print(f"X_test dataset Shape: {X_test.shape}")


X_train dataset Shape: (1460, 80)
y_train dataset Shape: (1460,)
X_test dataset Shape: (1459, 80)


# Missing Value imputation:

In [14]:
is_null_X_train = X_train.isnull().sum()
is_null_X_train


Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64


# For X_train

In [15]:

# for numerical values
num_var = X_train.select_dtypes(include=["int64","float64"])
num_var_miss = [var for var in num_var if is_null_X_train[var]>0]
num_var_miss


['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [18]:
#for categorical value
cat_var = X_train.select_dtypes(include=["object"]).keys()
cat_var_missing = [var for var in cat_var if is_null_X_train[var]>0]
cat_var_missing

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

<br><br>
আমরা ইচ্ছে করলেই  যে কোন column এ যে কোন strategy apply করতে পারি না । এর জন্য আমাদেরকে আগে ডাটা সর্ম্পকে ভালো ভাবে idea নিতে হয় । কিন্তু এখন আমরা সেইটা skip করতেছি । 
<br><br>

ধরি, <br>

'LotFrontage' এ mean value,<br>

'MasVnrArea', 'GarageYrBlt' এ median, <br>

'Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu', 
 এ most frequent বা mode,<br>

 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature' এ constant value  impute করবো <br>

In [20]:
num_var_mean = ['LotFrontage']

num_var_median = ['MasVnrArea', 'GarageYrBlt']

cat_var_mode = ['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu', ]

cat_var_constant = ['GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']
